In [27]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency
from itertools import combinations
from sklearn.neural_network import MLPClassifier
from numpy import random
import math

In [30]:
print("\nAdult dataset:")
z = pd.read_csv('/Users/HP/Downloads/adult.data')
row, col = z.shape
cols = z.columns
for c in cols:
    t = z.drop_duplicates(subset=c)
print(f'\tNo. of records: {row}')
print(f'\tNo. of incomplete records: {z.size - t.size}')
print(f'\tNo. of attributes: {col}')


Adult dataset:
	No. of records: 32560
	No. of incomplete records: 488370
	No. of attributes: 15


In [38]:
#Data Cleaning
f = pd.read_csv('/Users/HP/Downloads/adult.data')
# print(f)
d = np.array(f['39'])
# e = np.array(f['13'])
inc = np.array(f['39'])
dmax = np.max(d)
dmin = np.min(d)

In [39]:
print(d)
new_d = np.zeros(len(d))
nmax = 1
nmin = 0
r = (nmax-nmin)/(dmax-dmin)
for i in range(len(d)):
    new_d[i] = (d[i]-dmin)*r+nmin
print("\nMin Max Normalization:")
print(new_d)

#Z-score
new_z = np.zeros(len(d))
mu = 45
sd = 316.22
for i in range(len(d)):
    new_z[i] = (d[i]-mu)/sd
print("\nZ-score Normalization:")
print(new_z)

#Decimal-scaling
new_s = np.zeros(len(d))
#new range is [170...900]
for i in range(len(d)):
    new_s[i] = d[i]/0.1
print("\nDecimal Scaling Normalization:")
print(new_s)

[50 38 53 ... 58 22 52]

Min Max Normalization:
[0.45205479 0.28767123 0.49315068 ... 0.56164384 0.06849315 0.47945205]

Z-score Normalization:
[ 0.01581178 -0.02213649  0.02529884 ...  0.04111062 -0.07273417
  0.02213649]

Decimal Scaling Normalization:
[500. 380. 530. ... 580. 220. 520.]


In [47]:
#Bayes Classification
f = pd.read_csv('/Users/HP/Downloads/adult.data')
# f = f.head(20)
g = np.array(f['Male'])
inc = np.array(f['<=50K'])
df0 = pd.DataFrame((g,inc))
df0 = df0.T

arr = np.empty((2,2))

arr[0][0] = len(df0[ (df0[0] == 'Female') & (df[1] == '<=50K') ])
arr[0][1] = len(df0[ (df0[0] == 'Female') & (df[1] == '>50K') ])
arr[1][0] = len(df0[ (df0[0] == 'Male') & (df[1] == '<=50K') ])
arr[1][1] = len(df0[ (df0[0] == 'Male') & (df[1] == '>50K') ])

df1 = pd.DataFrame(arr,columns=['<=50K','>50K'],index=['Female','Male'])
print(df1)

p0 = df1.sum()[0]/df1.sum(1).sum()
p1 = df1.sum()[1]/df1.sum(1).sum()
pf = df1.sum(1)[0]/df1.sum(1).sum()
pm = df1.sum(1)[1]/df1.sum(1).sum()

pf0 = df1.iloc[0,0]/df1.sum()[0]
pf1 = df1.iloc[0,1]/df1.sum()[0]
pm0 = df1.iloc[1,0]/df1.sum()[0]
pm1 = df1.iloc[1,1]/df1.sum()[1]

def bayes(h, x):
    if(h == '<=50K'):
        if(x == 'Female'):
            return round((pf0*p0/pf),2)
        else:
            return round((pm0*p0/pm),2)
    else:
        if(x == 'Female'):
            return round((pf1*p1/pf),2)
        else:
            return round((pm1*p1/pm),2)

print(f'\nProbability of a Male having income <=50K is  {bayes("<=50K","Male")}')
print(f'\nProbability of a Female having income <=50K is  {bayes("<=50K","Female")}')
print(f'\nProbability of a Male having income >50K is  {bayes(">50K","Male")}')
print(f'\nProbability of a Female having income >50K is  {bayes(">50K","Female")}')

KeyError: 'Male'

In [48]:
df = pd.DataFrame(f)
df = df.drop_duplicates()


# f = f.head()
ip1 = np.array(df['White'])

ind = []
for i in range(len(ip1)):
    if(ip1[i] != "Black" and ip1[i] != "White"):
        ind.append(i)
        c+=1
ip1 = np.delete(ip1,ind)
ip1 = pd.Series(ip1)
ip1_map = {'Black':0,'White':1}
ip1 = ip1.map(ip1_map).astype(int)
ip2 = pd.Series(df['gender'])
ip2_map = {'Female':0,'Male':1}
ip2 = ip2.map(ip2_map).astype(int)
arr = np.array(list(zip(ip1,ip2)))


lr = 1 #learning rate
bias = 1 #value of bias
w = [random.random(),random.random(),random.random()]  #weights

def neu(x, y, op):
    outp = x*w[0] + y*w[1] + bias*w[2]
    if outp > 0 : #activation function
        outp = 1
    else :
        outp = 0
        error = op - outp
        w[0] += error * x * lr
        w[1]+= error * y * lr
        w[2] += error * bias * lr

for i in range(50):
    neu(0,0,0)
    neu(0,1,1)
    neu(1,0,1)
    neu(1,1,1)

pr = [0,0,1,1]
pg = [0,1,0,1]
for i,j in zip(pr,pg):
    neu(i,j,1)
    if(i == 0 and j == 0):
        if(outp == 0):
            print("A black female has income <=50K")
        else:
            print("A black female has income >50K")
    elif(i == 0 and j == 1):
        if(outp == 0):
            print("A black male has income <=50K")
        else:
            print("A black male has income >50K")
    elif(i == 1 and j == 0):
        if(outp == 0):
            print("A white female has income <=50K")
        else:
            print("A white female has income >50K")
    elif(i == 1 and j == 1):
        if(outp == 0):
            print("A white male has income <=50K")
        else:
            print("A white male has income >50K")
    

KeyError: 'White'

In [ ]:
#Linear 

import pandas as pd
import matplotlib.pyplot as plot
import numpy as np

In [35]:
data = pd.read_csv("/Users/HP/Downloads/adult.data")
data.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [49]:
x = data.age.to_numpy()
y = data.fnlwgt.to_numpy()
plot.scatter(x,y)

AttributeError: 'DataFrame' object has no attribute 'age'

In [ ]:
x = x/100.0
y= y/500000.0
plot.scatter(x,y)

In [ ]:
m,c = np.random.random(),np.random.random()
def learn(x,y,m,c,epoch):
    for i in range(epoch):
        error = y-(m*x+c)
        x_error = (y-(m*x+c))*x
        error = np.sum(error)/len(x)
        x_error = np.sum(x_error)/len(x)
        learning_rate = 0.01
        delta_m = learning_rate*x_error
        delta_c = learning_rate*error
        m+=delta_m
        c+=delta_c
    return(m,c)

m,c = learn(x,y,m,c,20000)
x1 = [0,1]
y1 = [0*m+c,1*m+c]

plot.scatter(x,y) 
plot.plot(x1,y1,'r')

In [ ]:
from sklearn.metrics import r2_score
print('R^2 :',r2_score(x1,y1))

In [ ]:
#Nonlinear
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
data = pd.read_csv("adult.csv")
%matplotlib inline

data.head()

In [ ]:
data.describe()

In [ ]:
def hrs(age,c0,c1,c2,c3):
    return c0+c1*age-c2*np.exp(-c3*age)
    
g = [50,0.08,50,0.08]

n = len(data["age"])
y = np.empty(n)
for i in range(n):
    y[i] = hrs(data["age"][i],g[0],g[1],g[2],g[3])
    
plt.plot(data["age"],data["hours-per-week"])
plt.plot(data["age"],y,"r.")

In [ ]:
#Logistic
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


data = pd.read_csv("adult.csv")

data.head()
plt.scatter(data.age,data["gender"],marker='+',color='red')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[['age']],data.sex,test_size=0.25)
X_train

In [ ]:
X_test

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,y_train)

In [ ]:
model.predict(X_test)

In [ ]:
model.score(X_test,y_test)

In [ ]:
model.predict_proba(X_test)

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def prediction_function(age):
    z = 0.04 * age - 2.73 
    y = sigmoid(z)
    return y

age = 50
prediction_function(age)

In [ ]:
age = 90
prediction_function(age)